TP1: Implementación de una red MLP para clasificación de imágenes de CIFAR 100

Hecho por: Tomás Castro

PASO 1: Abro todos los dataset de entrenamiento y prueba (ya se tantearon los datos brevemente en otra ocasión, así que en este cuaderno voy a ir directamente al entrenamiento)

In [ ]:
# Cargo todos los datasets
import numpy as np
import matplotlib.pyplot as plt
import pickle
%load_ext tensorboard

INPUT_DIR = "/kaggle/input/dl-itba-cifar-100-2024-q-1/"

x_train_ = np.load(INPUT_DIR+"x_train.npy")/255.
x_test = np.load(INPUT_DIR+"x_test.npy")/255.
y_train_coarse_ = np.load(INPUT_DIR+"y_train_coarse.npy")
y_train_fine_ = np.load(INPUT_DIR+"y_train_fine.npy")
with open(INPUT_DIR+"fine_label_names.pck", "rb") as f:
    labels_fine = pickle.load(f)
with open(INPUT_DIR+"coarse_label_names.pck", "rb") as f:
    labels_coarse = pickle.load(f)

PASO 2: Activo los TPU

In [ ]:
import tensorflow as tf
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)


PASO 3: Importo todas las funciones que voy a necesitar de keras y sklearn, además de que hago el augmentation sobre los datos de entrada.

Todos los callbacks fueron muy útiles. En concreto el reduceLRonPlateau ayudaba a afinar la optimización cuando el modelo se estancaba en un mismo punto, y el EarlyStopping ayudó a prevenir el overfitting, detectando cuando ya era imposible mejorar la precisión de validación. Se redujo la "paciencia" de 10 épocas a 5 épocas ya que se verificó que nunca hubo nuevo progreso cuando la optimización se estancaba en más de 5 etapas consecutivas. Debido al gran tamaño de las capas que se decidió adoptar (ver siguiente sección), se aumentó el batch size a 128.

NOTA: Como las imágenes son de baja resolución, no puedo variar demasiado el contenido original de las imágenes, de lo contrario la imagen pierde demasiada claridad (que incluso viéndola visualmente es muy baja y difícil en algunos casos determinar de qué se trata), lo que confundiría más a mi modelo.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Separación de datos en entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(x_train_, y_train_fine_, test_size=0.2, stratify=y_train_fine_)

#Augmentation
gen = ImageDataGenerator(
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        zoom_range=0.05,    
        rotation_range = 15,
        horizontal_flip=True)
flow = gen.flow(x_train, y_train, batch_size=128)

# Definición de callbacks
rlrop = ReduceLROnPlateau(
    monitor = "val_accuracy",
    factor = 0.5,
    patience = 3,
    verbose = 1,
    min_lr = 1e-5
)
mc = ModelCheckpoint(
    "best.weights.h5",
    monitor = "val_accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,
)
es = EarlyStopping(
    monitor = "val_accuracy",
    patience = 5, 
    verbose = 1,
    restore_best_weights = True,
)
tb = TensorBoard(
    log_dir="logs",
)

PASO 4: Defino mi arquitectura. Tras diversas pruebas, se establecieron cuatro capas densas, iniciando con la primera capa del tamaño igual a la de entrada y luego se va reduciendo de a 1024 neuronas (32*32) hasta llegar a 100 en la última que es el tamaño de la salida. Se intercalaron capas de Batch Normalization antes de poner la activación, con muy buenos resultados y acelerando notoriamente el entrenamiento. Inicialmente la precisión de entrenamiento era mucho más grande que la de validación lo que era un indicio de sobreadaptación, pero tras haber agregado el augmentation las precisiones se igualaron. 

Otras cosas que se intentaron y no funcionaron:
- Agregar regularizadores L1 y L2 en las capas densas: el modelo ajustaba peor. Comenzando con el parámetro de regularización en 0.001, la precisión en CV quedaba muy por debajo de la precisión en el entrenamiento. Posteriormente se fue disminuyendo hasta 0,000000000001. Si bien la diferencia entre ambos valores se mitigaba bastante y el modelo convergía bastante más rápido (alrededor de 15 épocas menos), la precisión en CV nunca llegó a superar 0,35, quedando siempre debajo del 0,37 que se obtuvo sin el regularizador.
- Restringir la norma de los pesos con max_norm. Lo único que se logró fue hacer más lento el proceso de convergencia y nunca superó el récord obtenido en CV.
- Agregar capas de Dropout luego de las de detección relu: se supone que debería reducir la brecha entre las precisiones de entrenamiento y validación, y si bien esto efectivamente se cumplió, el modelo tardó más en converger y la precisión máxima obtenida fue mucho menor (no superó 0,32 en CV).
- Agregar más capas densas al comienzo, de mayor tamaño: el entrenamiento era siempre más lento, tradaba más en converger y la precisión de CV quedaba considerablemente debajo de la de entrenamiento por overfitting.
- Modificar la tasa de aprendizaje: siempre se llegó a resultados finales peores, lo que indica que 0,001 es o está muy cerca de su valor óptimo. Tampoco mejoró la convergencia cambiar el tipo de optimizador a SGD o RMSprop.

In [ ]:
with strategy.scope():
    model = Sequential()
    model.add(Input(shape=(32,32,3)))
    model.add(Flatten())
    model.add(Dense(32*32*3, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(32*32*2, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(32*32, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(100, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    model.compile(loss="sparse_categorical_crossentropy", metrics = ["accuracy"], optimizer = Adam(learning_rate=0.001))
    model.summary()
    history = model.fit(
        flow,
        epochs=100, 
        validation_data=(x_val, y_val),
        callbacks= [
            rlrop,
            es,
            mc,
            tb,
        ]
    )

PASO 5: Plotear los valores de precisión, pérdidas y testear el modelo con los datos de prueba y exportar el resultado a csv.

Los resultados del entrenamiento no aparecerán en este cuaderno ya que debido a que necesitaba optimizar mis tiempos con otras materias hice la mayor cantidad de pruebas, sobre todo las últimas, mediante el comando save & commit para poder dejarlo corriendo en el servidor. De todas formas mostraré en este mismo repositorio imágenes con los gráficos de convergencia del entrenamiento que guardo en este mismo paso. Hasta la fecha, la mejor puntuación que hice en submit fue de 0,3636 con la configuración actual.

In [ ]:
plt.clf()
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend(loc="upper right")
plt.savefig("loss.png")

In [ ]:
plt.clf()
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.legend(loc="upper left")
plt.savefig("accuracy.png")

In [ ]:
import pandas as pd
predictions = model.predict(x_test).argmax(axis=1)
df = pd.DataFrame(predictions, columns=["Label"])
df.index.name = "Id"
df.to_csv("submission.csv")